# Task 6: FinTech Vendor Scorecard for Micro-Lending

In this task, we develop a Vendor Analytics Engine to score vendors for micro-lending opportunities. We combine NER-extracted entities and Telegram post metadata to calculate key business metrics and a final Lending Score for each vendor.

## Key Vendor Metrics
- **Posting Frequency:** Average number of posts per week
- **Average Views per Post:** Indicator of market reach
- **Top Performing Post:** Highest view count, product, and price
- **Average Price Point:** Business profile from NER
- **Lending Score:** Weighted combination of metrics

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load processed data (assume cleaned_messages.csv contains all posts with NER results)
df = pd.read_csv('../data/processed/cleaned_messages.csv', encoding='utf-8-sig')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

In [ ]:
import re

def extract_price(text):
    # Simple regex to extract price (e.g., 1000 ብር)
    match = re.search(r'(\d+[\,\.]?\d*)\s*ብር', str(text))
    if match:
        return float(match.group(1).replace(',', ''))
    return np.nan

def vendor_metrics(df, vendor):
    vendor_df = df[df['channel'] == vendor].copy()
    vendor_df = vendor_df.sort_values('timestamp')
    # Posting frequency (posts per week)
    if len(vendor_df) > 1:
        weeks = (vendor_df['timestamp'].max() - vendor_df['timestamp'].min()).days / 7
        posting_freq = len(vendor_df) / weeks if weeks > 0 else len(vendor_df)
    else:
        posting_freq = len(vendor_df)
    # Average views
    avg_views = vendor_df['views'].mean()
    # Top performing post
    top_post = vendor_df.loc[vendor_df['views'].idxmax()]
    top_product = top_post['message']
    top_price = extract_price(top_post['message'])
    # Average price point
    vendor_df['price'] = vendor_df['message'].apply(extract_price)
    avg_price = vendor_df['price'].mean()
    return {
        'posting_freq': posting_freq,
        'avg_views': avg_views,
        'top_product': top_product,
        'top_price': top_price,
        'avg_price': avg_price
    }

In [ ]:
# Calculate metrics and lending scores for all vendors
vendors = df['channel'].unique()
results = []
for vendor in vendors:
    metrics = vendor_metrics(df, vendor)
    # Example lending score: 0.5*avg_views + 0.5*posting_freq
    lending_score = 0.5 * (metrics['avg_views'] or 0) + 0.5 * (metrics['posting_freq'] or 0)
    metrics['vendor'] = vendor
    metrics['lending_score'] = lending_score
    results.append(metrics)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='lending_score', ascending=False)
display(results_df[['vendor', 'posting_freq', 'avg_views', 'avg_price', 'lending_score', 'top_product', 'top_price']])

## Summary

- Calculated key business and engagement metrics for each vendor
- Combined metrics into a simple Lending Score
- The scorecard helps EthioMart identify promising vendors for micro-lending

You can adjust metric weights or add more features for a more sophisticated scorecard.